<a href="https://colab.research.google.com/github/vbloise3/Data_Science/blob/master/GettingData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
import sys, re
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Download a file based on its file ID.
#
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz
#file_id = 'REPLACE_WITH_YOUR_FILE_ID'
#downloaded = drive.CreateFile({'id': file_id})
#print('Downloaded content "{}"'.format(downloaded.GetContentString()))

# list files 
listed_txt = drive.ListFile({'q': "title contains '.txt' and 'root' in parents"}).GetList()
listed_ipynb = drive.ListFile({'q': "'1C9uuMFnTnllNbyaUWhsHk-PK0bDMpMuQ' in parents"}).GetList()
for file in listed_txt:
  print('title {}. id {}'.format(file['title'], file['id']))
for file in listed_ipynb:
  print('title {}. id {}'.format(file['title'], file['id']))

# sys.argv is the list of command-line arguments 
# sys.argv[ 0] is the name of the program itself 
# sys.argv[ 1] will be the regex specified at the command line 
# run it like this: cat SomeFile.txt | python egrep.py "[ 0-9]" | python line_count.py
regex = sys.argv[ 1]
some_lines = ["one line ", "two lines ", "three! "]

# for every line passed into the script 
for line in sys.stdin: 
  # if it matches the regex, write it to stdout 
  if re.search( regex, line): 
    sys.stdout.write( line)
for line in some_lines:
  sys.stdout.write(line + "\n")

# email address domain stripper exaMPLE
def get_domain( email_address: str)-> str: 
  """ Split on '@' and return the last piece""" 
  return email_address.lower(). split("@")[-1]

# a couple of tests 
assert get_domain(' joelgrus@gmail.com') == 'gmail.com'
assert get_domain(' joel@m.datasciencester.com') == 'm.datasciencester.com'

#with open(' email_addresses.txt', 'r') as f: 
#  domain_counts = Counter( get_domain( line.strip()) 
#  for line in f if "@" in line)


title data.txt. id 1yIcwI0LJVpw8rAel9DA8qcJeEfjllkun
title GettingData.ipynb. id 1nKZv3XGsSPtW1WfrirQKRgWQ14s8mp8m
title GradientDescent.ipynb. id 1OL130LGnKNLk9BbzbRekiaTOVBNPc903
title PythonWorkoutChapter1.ipynb. id 1hQ_qWfn_tj8KBQspsqsIJQh8j_dqFq1e
title HypothesisandInference.ipynb. id 1GngAYwLNtVi9kntFYbivstv_XoY6pP8q
title Probability.ipynb. id 1SF6aug-J0inOZTWytKpr6F7s9Bf3nGO2
title Statistics.ipynb. id 1FgvOUIxRhmt5J3KTj_CkTjEkx7W4Pvdz
title Copy of Linear Algebra.ipynb. id 1nbLcny5T4AgdjB6fFa0oVrEq8UVx7u9r
title Linear Algebra.ipynb. id 1az-uba8FteC7F1VbBXPCJYUHvUySKjLd
title firstNN.ipynb. id 1m5fbJPk_nUkXUU1wzV5MmPBzX0Ztum75
title testNN.ipynb. id 1M-yjGIWvGrS7muzi6-OHXPkSho0C2VKH
one line 
two lines 
three! 
